# Team 7 

### Beaula Mahima V - MA21BTECH11002
### Riya Ann Easow - EE21BTECH11044
### Kallu Rithika - AI22BTECH11010
### Prasham Walvekar - CS21BTECH11047

In [ ]:
import os
import numpy as np
from PIL import Image

csv_filename = "metrics_cGAN.csv"
output_dir = "output_dir"
input_dir = "input_dir"
original_img_paths = [os.path.join(input_dir, file_name) for file_name in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, file_name))]
#original input that is assumed to be coloured

def load_images_from_directory(path):
    """
    Load all images from a directory into a list.

    Args:
        path (str): The path to the directory containing images.

    Returns:
        list: A list of PIL.Image objects.
    """
    images = []
    for file_name in os.listdir(path):
        file_path = os.path.join(path, file_name)
        if os.path.isfile(file_path):
            try:
                img = Image.open(file_path)
                img = np.array(img)
                images.append(img)
            except Exception as e:
                print(f"Error loading image {file_name}: {e}")
    return images

colorized_images = load_images_from_directory(output_dir)

### SSIM

In [ ]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm

# --- Define SSIM Function ---
def compute_ssim(original, colorized):
    """Computes SSIM between two color images with the size of colorized image."""
    height, width, _ = colorized.shape  # Get dimensions of the colorized image
    original_resized = cv2.resize(original, (width, height), interpolation=cv2.INTER_CUBIC)
    
    return ssim(original_resized, colorized, data_range=255, channel_axis=2)

# --- Compute SSIM for All Images ---
ssim_values = []
batch_size = 64  # Number of images per batch

for i in tqdm(range(0, len(original_img_paths), batch_size), desc="Processing Batches", unit="batch"):
    batch_original = original_img_paths[i:i+batch_size]
    batch_colorized = colorized_images[i:i+batch_size]
    
    for orig_path, colorized_rgb in zip(batch_original, batch_colorized):
        try:
            original_rgb = cv2.imread(orig_path)  # Load original image
            original_rgb = cv2.cvtColor(original_rgb, cv2.COLOR_BGR2RGB)  # Convert to RGB
            
            # Compute SSIM
            ssim_val = compute_ssim(original_rgb, colorized_rgb)
            ssim_values.append((orig_path, ssim_val))
        except Exception as e:
            print(f"Skipping {orig_path} due to error: {e}")
            ssim_values.append((orig_path, np.nan))

# --- Save SSIM Results to a New CSV ---
ssim_df = pd.DataFrame(ssim_values, columns=["Image Path", "SSIM"])
ssim_df.to_csv(csv_filename, index=False)
print(f"\nSSIM results saved to {csv_filename}")

# --- Compute Summary Statistics ---
summary_stats = ssim_df["SSIM"].describe()
print("\nSSIM Summary Statistics:")
print(summary_stats)

# --- Plot Histogram for SSIM ---
plt.figure(figsize=(8, 6))
plt.hist(ssim_df["SSIM"].dropna(), bins=30, alpha=0.75, color='b', edgecolor='black')
plt.title("Histogram of SSIM")
plt.xlabel("SSIM")
plt.ylabel("Frequency")
plt.savefig("ssim_histogram.png")
plt.show()
print("\nHistogram saved as ssim_histogram.png")


### PCQI

In [ ]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
from scipy.ndimage import gaussian_filter
import torchvision.transforms as transforms
from PIL import Image
import os

# --- Define PCQI Function ---
def compute_pcqi(original, colorized, sigma=1.5):
    """Computes PCQI (Perception-based Contrast Quality Index) for color images."""

    # Convert images to YIQ color space
    original_yiq = cv2.cvtColor(original, cv2.COLOR_RGB2YCrCb)
    colorized_yiq = cv2.cvtColor(colorized, cv2.COLOR_RGB2YCrCb)

    # Extract luminance (Y) and chrominance (I, Q) channels
    Y_orig, I_orig, Q_orig = cv2.split(original_yiq)
    Y_col, I_col, Q_col = cv2.split(colorized_yiq)

    # --- Compute Structural Similarity Index (SSIM) for Luminance ---
    luminance_similarity = ssim(Y_orig, Y_col, data_range=Y_orig.max() - Y_orig.min())

    # --- Compute Chrominance Similarity ---
    def chroma_similarity(channel_orig, channel_col):
        mu_x = gaussian_filter(channel_orig, sigma)
        mu_y = gaussian_filter(channel_col, sigma)
        sigma_x = np.sqrt(gaussian_filter(channel_orig**2, sigma) - mu_x**2)
        sigma_y = np.sqrt(gaussian_filter(channel_col**2, sigma) - mu_y**2)
        contrast_measure = (2 * sigma_x * sigma_y) / (sigma_x**2 + sigma_y**2 + 1e-6)
        structure_measure = (2 * mu_x * mu_y) / (mu_x**2 + mu_y**2 + 1e-6)
        return np.mean(contrast_measure * structure_measure)

    I_similarity = chroma_similarity(I_orig, I_col)
    Q_similarity = chroma_similarity(Q_orig, Q_col)
    chrominance_similarity = (I_similarity + Q_similarity) / 2

    # --- Final PCQI Score ---
    pcqi_score = 0.5 * luminance_similarity + 0.5 * chrominance_similarity
    return pcqi_score


if os.path.exists(csv_filename):
    df = pd.read_csv(csv_filename)  # Load existing file
else:
    df = pd.DataFrame(columns=["Image Path"])  # Create new if not found

# --- Compute PCQI for All Images in Batches ---
batch_size = 10  # Number of images per batch
new_metrics = {}

for i in tqdm(range(0, len(original_img_paths), batch_size), desc="Processing Batches", unit="batch"):
    batch_original = original_img_paths[i:i+batch_size]
    batch_colorized = colorized_images[i:i+batch_size]

    for orig_path, colorized_rgb in zip(batch_original, batch_colorized):
        original_rgb = Image.open(orig_path).convert("RGB")  # Load original grayscale image as RGB
        colorized_rgb = Image.fromarray(colorized_rgb)

        # Get dimensions of the colorized image
        width, height = colorized_rgb.size
        transform = transforms.Compose([
            transforms.Resize((height, width), Image.BICUBIC),
            transforms.ToTensor()
        ])

        # Resize images using the defined transform
        original_rgb = transform(original_rgb).permute(1, 2, 0).numpy()
        colorized_rgb = transform(colorized_rgb).permute(1, 2, 0).numpy()

        # Convert images back to uint8 for OpenCV functions
        original_rgb = (original_rgb * 255).astype(np.uint8)
        colorized_rgb = (colorized_rgb * 255).astype(np.uint8)

        pcqi_val = compute_pcqi(original_rgb, colorized_rgb)
        new_metrics[orig_path] = pcqi_val

# --- Update DataFrame with PCQI Scores ---
df["PCQI"] = df["Image Path"].map(new_metrics)  # Map new values to existing entries

# --- Save Updated CSV ---
df.to_csv(csv_filename, index=False)
print(f"\nUpdated CSV saved as {csv_filename}")

# --- Compute and Display Summary Statistics ---
summary_stats = df.drop(columns=["Image Path"]).describe().T
print("\nSummary Statistics:")
print(summary_stats)

# --- Plot Histogram for PCQI Scores ---
plt.figure(figsize=(8, 6))
plt.hist(df["PCQI"].dropna(), bins=30, alpha=0.75, color='g', edgecolor='black')
plt.title("Histogram of PCQI Scores")
plt.xlabel("PCQI")
plt.ylabel("Frequency")
plt.savefig("colorization_metrics_histogram_pcqi.png")
plt.show()
print("\nHistogram saved as colorization_metrics_histogram_pcqi.png")


### Colorfulness Index

In [ ]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- Define Metric Function ---
def compute_colorfulness(image):
    """Computes the Colorfulness Index (CI)."""
    (R, G, B) = cv2.split(image)
    rg = np.abs(R - G)
    yb = np.abs(0.5 * (R + G) - B)
    std_rg, std_yb = np.std(rg), np.std(yb)
    mean_rg, mean_yb = np.mean(rg), np.mean(yb)
    return np.sqrt(std_rg**2 + std_yb**2) + (0.3 * np.sqrt(mean_rg**2 + mean_yb**2))


try:
    df = pd.read_csv(csv_filename)  # Load existing CSV
except FileNotFoundError:
    df = pd.DataFrame(columns=["Image Path"])  # Create new if not found

# --- Compute Colorfulness for Each Image ---
batch_size = 64  # Number of images per batch
new_metrics = {}

for i in tqdm(range(0, len(original_img_paths), batch_size), desc="Processing Batches", unit="batch"):
    batch_original = original_img_paths[i:i+batch_size]
    batch_colorized = colorized_images[i:i+batch_size]

    for orig_path, colorized_rgb in zip(batch_original, batch_colorized):
        # Get dimensions of the colorized image
        height, width, _ = colorized_rgb.shape
        
        # Resize the original image to match colorized image size
        original_rgb = cv2.imread(orig_path)
        original_rgb = cv2.cvtColor(original_rgb, cv2.COLOR_BGR2RGB)
        original_rgb = cv2.resize(original_rgb, (width, height), interpolation=cv2.INTER_CUBIC)
        
        ci = compute_colorfulness(colorized_rgb)
        new_metrics[orig_path] = ci

# --- Update the Existing DataFrame ---
df["Colorfulness"] = df["Image Path"].map(new_metrics)  # Map new values

# --- Save Updated CSV ---
df.to_csv(csv_filename, index=False)
print(f"\nUpdated CSV saved as {csv_filename}")

# --- Compute and Display Summary Statistics ---
summary_stats = df.drop(columns=["Image Path"]).describe().T
print("\nSummary Statistics:")
print(summary_stats)

# --- Plot Histogram for Colorfulness ---
plt.figure(figsize=(8, 6))
plt.hist(df["Colorfulness"].dropna(), bins=30, alpha=0.75, color='b', edgecolor='black')
plt.title("Histogram of Colorfulness")
plt.xlabel("Colorfulness")
plt.ylabel("Frequency")
plt.savefig("colorization_colorfulness_histogram.png")
plt.show()
print("\nHistogram saved as colorization_colorfulness_histogram.png")


### Color Harmony

In [ ]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- Define Metric Function ---
def compute_color_harmony(image):
    """Computes a color harmony score based on hue variance."""
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    hue = hsv[:, :, 0].flatten()
    hue_variance = np.var(hue)
    return 1 / (1 + hue_variance)


try:
    df = pd.read_csv(csv_filename)  # Load existing CSV
except FileNotFoundError:
    df = pd.DataFrame(columns=["Image Path"])  # Create new if not found

# --- Compute Color Harmony for Each Image ---
batch_size = 64  # Number of images per batch
new_metrics = {}

for i in tqdm(range(0, len(original_img_paths), batch_size), desc="Processing Batches", unit="batch"):
    batch_original = original_img_paths[i:i+batch_size]
    batch_colorized = colorized_images[i:i+batch_size]

    for img_path, colorized_rgb in zip(batch_original, batch_colorized):
        harmony = compute_color_harmony(colorized_rgb)
        new_metrics[img_path] = harmony

# --- Update the Existing DataFrame ---
df["Color Harmony"] = df["Image Path"].map(new_metrics)  # Map new values

# --- Save Updated CSV ---
df.to_csv(csv_filename, index=False)
print(f"\nUpdated CSV saved as {csv_filename}")

# --- Compute and Display Summary Statistics ---
summary_stats = df.drop(columns=["Image Path"]).describe().T
print("\nSummary Statistics:")
print(summary_stats)

# --- Plot Histogram for Color Harmony ---
plt.figure(figsize=(6, 6))
plt.hist(df["Color Harmony"].dropna(), bins=30, alpha=0.75, color='g', edgecolor='black')
plt.title("Histogram of Color Harmony")
plt.xlabel("Color Harmony")
plt.ylabel("Frequency")
plt.savefig("colorization_metrics_histogram.png")
plt.show()
print("\nHistogram saved as colorization_metrics_histogram.png")


### Color Distribution Balance

In [ ]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- Define Metric Function ---
def compute_color_distribution_balance(image, num_bins=32):
    """Computes color distribution balance using histogram entropy."""
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    h_hist = cv2.calcHist([hsv], [0], None, [num_bins], [0, 180])
    h_hist = h_hist / np.sum(h_hist)  # Normalize histogram
    return 1 - np.sum(h_hist * np.log(h_hist + 1e-6)) / np.log(num_bins)


try:
    df = pd.read_csv(csv_filename)  # Load existing CSV
except FileNotFoundError:
    df = pd.DataFrame(columns=["Image Path"])  # Create new if not found

# --- Compute Color Distribution Balance for Each Image ---
batch_size = 64  # Number of images per batch
new_metrics = {}

for i in tqdm(range(0, len(original_img_paths), batch_size), desc="Processing Batches", unit="batch"):
    batch_original = original_img_paths[i:i+batch_size]
    batch_colorized = colorized_images[i:i+batch_size]

    for img_path, colorized_rgb in zip(batch_original, batch_colorized):
        balance = compute_color_distribution_balance(colorized_rgb)
        new_metrics[img_path] = balance

# --- Update the Existing DataFrame ---
df["Color Distribution Balance"] = df["Image Path"].map(new_metrics)  # Map new values

# --- Save Updated CSV ---
df.to_csv(csv_filename, index=False)
print(f"\nUpdated CSV saved as {csv_filename}")

# --- Compute and Display Summary Statistics ---
summary_stats = df.drop(columns=["Image Path"]).describe().T
print("\nSummary Statistics:")
print(summary_stats)

# --- Plot Histogram for Color Distribution Balance ---
plt.figure(figsize=(6, 6))
plt.hist(df["Color Distribution Balance"].dropna(), bins=30, alpha=0.75, color='b', edgecolor='black')
plt.title("Histogram of Color Distribution Balance")
plt.xlabel("Color Distribution Balance")
plt.ylabel("Frequency")
plt.savefig("colorization_metrics_histogram.png")
plt.show()
print("\nHistogram saved as colorization_metrics_histogram.png")
